TITLE HERE!

In [61]:
# Import thư viện cần thiết
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import re


In [62]:
# Đường dẫn thư mục chứa các file parquet
base_path = Path(r"D:\recommendation dataset")

In [63]:
# Tắt cảnh báo không cần thiết
warnings.filterwarnings('ignore')

# Cài đặt style cho biểu đồ
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [64]:
# Tìm tất cả file parquet trong thư mục
all_parquet_files = list(base_path.glob("*.parquet"))
print(f"Tìm thấy {len(all_parquet_files)} file parquet trong thư mục.")

# Phân loại file theo tên
sales_item_files = []
sales_purchase_files = []
sales_user_files = []

for file_path in all_parquet_files:
    name = file_path.name.lower()
    
    if "sales_pers.item" in name:
        sales_item_files.append(file_path)
    elif "sales_pers.purchase" in name or "sales_pers.purchase_history_daily" in name:
        sales_purchase_files.append(file_path)
    elif "sales_pers.user" in name:
        sales_user_files.append(file_path)

# In kết quả phân loại
print(f"""
 Tổng kết:
   sales_pers.item: {len(sales_item_files)} files
   sales_pers.purchase: {len(sales_purchase_files)} files
   sales_pers.user: {len(sales_user_files)} files
""")

# Hàm đọc nhóm file parquet
def read_parquet_group(file_list, group_name):
    if not file_list:
        print(f"Không có file nào cho nhóm {group_name}")
        return None
    
    print(f"Đang đọc {len(file_list)} file(s) cho nhóm {group_name}...")
    
    df = pl.read_parquet(file_list)
    
    print(f"{group_name} loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
    return df

# Đọc dữ liệu từ các nhóm file
sales_item_df = read_parquet_group(sales_item_files, "SALES ITEM")
sales_purchase_df = read_parquet_group(sales_purchase_files, "SALES PURCHASE")
sales_user_df = read_parquet_group(sales_user_files, "SALES USER")

# Tổng kết dữ liệu đã đọc
print("\nTỔNG KẾT DỮ LIỆU:")
if sales_item_df is not None:
    print(f"sales_item_df: {sales_item_df.shape}")
if sales_purchase_df is not None:
    print(f"sales_purchase_df: {sales_purchase_df.shape}")
if sales_user_df is not None:
    print(f"sales_user_df: {sales_user_df.shape}")

Tìm thấy 83 file parquet trong thư mục.

 Tổng kết:
   sales_pers.item: 1 files
   sales_pers.purchase: 72 files
   sales_pers.user: 10 files

Đang đọc 1 file(s) cho nhóm SALES ITEM...
SALES ITEM loaded: 27,332 rows × 34 columns
Đang đọc 72 file(s) cho nhóm SALES PURCHASE...
SALES PURCHASE loaded: 35,729,825 rows × 16 columns
Đang đọc 10 file(s) cho nhóm SALES USER...
SALES USER loaded: 4,573,964 rows × 18 columns

TỔNG KẾT DỮ LIỆU:
sales_item_df: (27332, 34)
sales_purchase_df: (35729825, 16)
sales_user_df: (4573964, 18)


In [65]:
# Tạo dictionary chứa các DataFrame
datasets = {
    "SALES ITEM": sales_item_df,
    "SALES PURCHASE": sales_purchase_df,
    "SALES USER": sales_user_df
}
# Lặp qua từng nhóm dữ liệu trong dictionary 'datasets'
for name, df in datasets.items():
    
    # Nếu không có dữ liệu (None), bỏ qua nhóm đó
    if df is None:
        continue

    # In tiêu đề nhóm dữ liệu
    print(f"\n[ {name} ]")

    # In kích thước của DataFrame: số dòng và số cột
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

    # In danh sách tên các cột
    print("Columns:", df.columns)

    # Kiểm tra số lượng giá trị thiếu (null) trong từng cột
    print("\nMissing values:")
    null_counts = df.null_count().to_pandas().T
    print(null_counts)

    # Mô tả thống kê cho các cột số
    print("\nMô tả dữ liệu số:")
    display(df.describe())


[ SALES ITEM ]
Shape: 27,332 rows × 34 columns
Columns: ['p_id', 'item_id', 'price', 'category_l1_id', 'category_l1', 'category_l2_id', 'category_l2', 'category_l3_id', 'category_l3', 'category_id', 'category', 'description', 'brand', 'manufacturer', 'creation_timestamp', 'is_deleted', 'created_date', 'updated_date', 'sync_status_id', 'last_sync_date', 'sync_error_message', 'image_url', 'gender_target', 'age_group', 'item_type', 'gp', 'weight', 'color', 'size', 'origin', 'volume', 'material', 'sale_status', 'description_new']

Missing values:
                        0
p_id                    0
item_id                 0
price                   0
category_l1_id          0
category_l1             0
category_l2_id          0
category_l2             0
category_l3_id          0
category_l3             0
category_id             0
category                0
description             0
brand                   0
manufacturer            0
creation_timestamp      0
is_deleted              0
created_

statistic,p_id,item_id,price,category_l1_id,category_l1,category_l2_id,category_l2,category_l3_id,category_l3,category_id,category,description,brand,manufacturer,creation_timestamp,is_deleted,created_date,updated_date,sync_status_id,last_sync_date,sync_error_message,image_url,gender_target,age_group,item_type,gp,weight,color,size,origin,volume,material,sale_status,description_new
str,f64,str,f64,f64,str,f64,str,f64,str,f64,str,str,str,str,f64,f64,str,str,f64,str,str,str,str,str,str,f64,f64,str,str,str,str,str,f64,str
"""count""",27332.0,"""27332""",27332.0,27332.0,"""27332""",27332.0,"""27332""",27332.0,"""27332""",27332.0,"""27332""","""27332""","""27332""","""27332""",27332.0,27332.0,"""27332""","""27332""",26546.0,"""26546""","""0""","""27332""","""27332""","""27332""","""27332""",27332.0,0.0,"""27332""","""27332""","""27332""","""27332""","""27332""",27332.0,"""22317"""
"""null_count""",0.0,"""0""",0.0,0.0,"""0""",0.0,"""0""",0.0,"""0""",0.0,"""0""","""0""","""0""","""0""",0.0,0.0,"""0""","""0""",786.0,"""786""","""27332""","""0""","""0""","""0""","""0""",0.0,27332.0,"""0""","""0""","""0""","""0""","""0""",0.0,"""5015"""
"""mean""",106317.679643,null,190456.829467,2836.189192,null,3719.039843,null,4876.768294,null,4920.712718,null,null,null,null,1.6225e9,0.0,"""2021-05-31 17:17:33.849833""","""2025-08-30 04:07:07.766157""",2.0,"""2025-07-18 18:00:25.086391""",null,null,null,null,null,49696.651226,null,null,null,null,null,null,0.167203,null
"""std""",25425.734073,null,511123.971813,1417.521137,null,1935.042044,null,2035.952741,null,1999.308355,null,null,null,null,7.5666e7,null,null,null,0.0,null,null,null,null,null,null,144155.68753,null,null,null,null,null,null,0.373164,null
"""min""",1675.0,"""0000280000019""",0.0,1.0,"""Babycare""",2.0,"""0-1Y""",14.0,"""A2 milk""",4.0,"""0-12M Bodysuit bé trai đùi""","""Với kiểu dáng đơn giản nhưng …","""137 Degrees""","""(L) Địa điểm kinh doanh 1 - Ch…",1.2983e9,0.0,"""2011-02-21 11:54:02.047000""","""2023-12-26 17:21:11.663000""",2.0,"""2025-07-18 17:59:29.898256""",null,"""Không xác định""","""Bé Gái""","""0-10M""","""BCS""",0.0,null,"""Bạc""","""0 tháng""",""" Singapore, Thụy Sĩ""","""1 lít""",""" Vải không dệt, bông cellulose…",0.0,"""Chi tiết sản phẩm …"
"""25%""",88905.0,null,49000.0,2788.0,null,2038.0,null,3516.0,null,3522.0,null,null,null,null,1.5755e9,null,"""2019-12-04 18:24:16.260000""","""2025-08-18 09:59:19.847000""",2.0,"""2025-07-18 17:59:29.898256""",null,null,null,null,null,0.0,null,null,null,null,null,null,0.0,null
"""50%""",104510.0,null,119000.0,3292.0,null,3958.0,null,5254.0,null,5511.0,null,null,null,null,1.6358e9,null,"""2021-11-01 13:19:26.080000""","""2025-09-26 08:05:23.153000""",2.0,"""2025-07-18 17:59:29.898256""",null,null,null,null,null,25047.0,null,null,null,null,null,null,0.0,null
"""75%""",120262.0,null,199000.0,3292.0,null,4571.0,null,6984.0,null,6988.0,null,null,null,null,1.6699e9,null,"""2022-12-01 15:40:42.260000""","""2025-09-27 00:05:36.233000""",2.0,"""2025-07-18 17:59:29.898256""",null,null,null,null,null,65532.0,null,null,null,null,null,null,0.0,null
"""max""",167147.0,"""7497000000006""",2.099e7,7376.0,"""Đồ chơi & Sách""",7492.0,"""Đồ uống""",7493.0,"""Đồ hộp""",7501.0,"""Động vật mô hình""","""﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿Bỉm …","""Đức Thành""","""﻿Shantou City Chenghai Jia Ron…",1.7559e9,0.0,"""2025-08-22 17:35:14.377000""","""2025-10-01 08:05:42.990000""",2.0,"""2025-07-21 10:59:04.045058""",null,"""Không xác định""","""Unisex""","""[""Từ 6M"", ""Từ 9M"", ""Từ 1Y""]""","""Ứng dụng mẹ và bé""",5.54136e6,null,"""Đỏ""","""XXL(15-25kg) - 26 miếng""","""Ấn Độ, Bulgaria""","""hộp 3 lọ x 250ml""","""﻿100% cotton﻿<br>""",1.0,"""Xin lỗi, nhưng tôi không thể t…"



[ SALES PURCHASE ]
Shape: 35,729,825 rows × 16 columns
Columns: ['timestamp', 'user_id', 'item_id', 'event_type', 'event_value', 'price', 'date_key', 'quantity', 'customer_id', 'created_date', 'updated_date', 'channel', 'payment', 'location', 'discount', 'is_deleted']

Missing values:
              0
timestamp     0
user_id       0
item_id       0
event_type    0
event_value   0
price         0
date_key      0
quantity      0
customer_id   0
created_date  0
updated_date  0
channel       0
payment       0
location      0
discount      0
is_deleted    0

Mô tả dữ liệu số:


statistic,timestamp,user_id,item_id,event_type,event_value,price,date_key,quantity,customer_id,created_date,updated_date,channel,payment,location,discount,is_deleted
str,f64,str,str,str,f64,f64,f64,f64,f64,str,str,str,str,f64,f64,f64
"""count""",3.5729825e7,"""35729825""","""35729825""","""35729825""",3.5729825e7,3.5729825e7,3.5729825e7,3.5729825e7,3.5729825e7,"""35729825""","""35729825""","""35729825""","""35729825""",3.5729825e7,3.5729825e7,3.5729825e7
"""null_count""",0.0,"""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,"""0""","""0""","""0""","""0""",0.0,0.0,0.0
"""mean""",1.7199e9,null,null,null,1.508098,169188.742494,2.0241e7,1.508098,4.8713e6,"""2024-07-02 17:13:24.220955""","""2024-07-02 18:27:59.561504""",null,null,448.414508,17235.651914,0.0
"""std""",9.1568e6,null,null,null,2.264976,195539.88976,346.231465,2.264976,2.2697e6,null,null,null,null,247.505432,46673.708499,null
"""min""",1.7041e9,"""0000063a29e92d643437aa09266fdc…","""0000280000019""","""Purchase""",1.0,0.0444,2.0240101e7,1.0,14732.0,"""2024-01-01 06:44:59.037000""","""2024-01-01 06:44:59.233000""","""Android""","""Chuyển khoản""",16.0,0.0,0.0
"""25%""",1.7120e9,null,null,null,1.0,48316.3914,2.0240401e7,1.0,3.06641e6,"""2024-04-01 16:44:10.513000""","""2024-04-01 17:15:12.017000""",null,null,234.0,0.0,null
"""50%""",1.7199e9,null,null,null,1.0,87000.0,2.0240702e7,1.0,5.373736e6,"""2024-07-02 19:06:52.293000""","""2024-07-02 19:42:24.550000""",null,null,439.0,0.0,null
"""75%""",1.7279e9,null,null,null,1.0,244000.0,2.0241002e7,1.0,6.853734e6,"""2024-10-02 19:55:22.447000""","""2024-10-02 20:24:53.813000""",null,null,653.0,15000.0,null
"""max""",1.7357e9,"""fffff7e197ebf5c523fa686e6a305c…","""7352000000004""","""Purchase""",240.0,2.099e7,2.0241231e7,240.0,8.208027e6,"""2024-12-31 22:35:19.510000""","""2025-09-27 21:25:22.960000""","""iOS""","""ZaloPay""",1036.0,7.995e6,0.0



[ SALES USER ]
Shape: 4,573,964 rows × 18 columns
Columns: ['customer_id', 'gender', 'location', 'province', 'membership', 'timestamp', 'created_date', 'updated_date', 'sync_status_id', 'last_sync_date', 'sync_error_message', 'region', 'location_name', 'install_app', 'install_date', 'district', 'user_id', 'is_deleted']

Missing values:
                          0
customer_id               0
gender                    0
location                  0
province                  0
membership                0
timestamp                 0
created_date              0
updated_date              0
sync_status_id       338285
last_sync_date       338285
sync_error_message  4573964
region                    0
location_name             0
install_app               0
install_date              0
district                  0
user_id                   0
is_deleted                0

Mô tả dữ liệu số:


statistic,customer_id,gender,location,province,membership,timestamp,created_date,updated_date,sync_status_id,last_sync_date,sync_error_message,region,location_name,install_app,install_date,district,user_id,is_deleted
str,f64,str,f64,str,str,f64,str,str,f64,str,str,str,str,str,f64,str,str,f64
"""count""",4.573964e6,"""4573964""",4.573964e6,"""4573964""","""4573964""",4.573964e6,"""4573964""","""4573964""",4.235679e6,"""4235679""","""0""","""4573964""","""4573964""","""4573964""",4.573964e6,"""4573964""","""4573964""",4.573964e6
"""null_count""",0.0,"""0""",0.0,"""0""","""0""",0.0,"""0""","""0""",338285.0,"""338285""","""4573964""","""0""","""0""","""0""",0.0,"""0""","""0""",0.0
"""mean""",6.1266e6,null,554.053518,null,null,1.6712e9,"""2022-12-16 22:48:44.025646""","""2025-06-11 05:52:40.770554""",2.0,"""2025-07-16 15:20:34.188640""",null,null,null,null,1.6841e9,null,null,0.0
"""std""",2.4412e6,null,295.523361,null,null,7.9743e7,null,null,0.0,null,null,null,null,null,7.1374e7,null,null,null
"""min""",14732.0,"""Khác""",42.0,"""An Giang""","""Diamond""",1.3064e9,"""2011-05-25 21:11:51.677000""","""2019-06-30 08:38:47.100000""",2.0,"""2025-07-16 11:54:29.816986""",null,"""Bắc Trung Bộ""","""AGI - 110 Quốc Lộ 91""","""Android""",1.3063e9,""" Vũng Liêm""","""000004697bc1f0ecab42b6a3c2f0bc…",0.0
"""25%""",4.53268e6,null,302.0,null,null,1.6343e9,"""2021-10-15 10:09:34.280000""","""2025-07-07 15:33:10.201316""",2.0,"""2025-07-16 11:58:58.022193""",null,null,null,null,1.6550e9,null,null,null
"""50%""",6.843955e6,null,547.0,null,null,1.6960e9,"""2023-09-29 16:33:25.217000""","""2025-07-07 15:33:10.201316""",2.0,"""2025-07-16 12:03:34.550186""",null,null,null,null,1.7054e9,null,null,null
"""75%""",8.029649e6,null,765.0,null,null,1.7312e9,"""2024-11-09 14:38:22.223000""","""2025-08-02 10:54:03.860000""",2.0,"""2025-07-16 12:11:53.888687""",null,null,null,null,1.7366e9,null,null,null
"""max""",9.264159e6,"""Nữ""",1310.0,"""Đồng Tháp""","""Standard""",1.7593e9,"""2025-09-30 22:00:36.623000""","""2025-09-30 23:22:08.957000""",2.0,"""2025-07-21 11:19:29.058389""",null,"""Đồng bằng sông Cửu Long""","""ĐTH - Trần Hưng Đạo""","""iOS""",1.7592e9,"""Ứng Hòa""","""fffffbd469d500229dd000e9fc48ed…",0.0


TASK 1:

In [74]:
# LOẠI BỎ CÁC CỘT KHÔNG CẦN THIẾT

# Danh sách các cột cần loại bỏ
cols_to_drop = [
    # SALES ITEM
    "is_deleted",
    "last_sync_date",
    "sync_error_message",
    "image_url",
    "description_new",
    "weight",
    "sync_status_id",
    "p_id",
    "item_id",
    "category_l1_id",
    "category_l2_id",
    "category_l3_id",
    "category_id",
    "description",
    "manufacturer",
    "created_date",
    "updated_date",

    # SALES PURCHASE
    "is_deleted",
    "event_type",
    "user_id",
    "item_id",
    "customer_id",
    "created_date",
    "updated_date",

    # SALES USER
    "is_deleted",
    "sync_status_id",
    "last_sync_date",
    "sync_error_message",
    "customer_id",
    "user_id",
    "created_date",
    "updated_date"

]

print("\nBẮT ĐẦU LOẠI BỎ CÁC CỘT KHÔNG CẦN THIẾT...\n")

# Lặp qua từng DataFrame trong datasets
for name, df in datasets.items():
    if df is None:
        continue

    print(f"\n>>> {name}")
    existing_cols = [c for c in cols_to_drop if c in df.columns]

    if existing_cols:
        print(f" - Các cột sẽ bị loại bỏ: {existing_cols}")
        df = df.drop(existing_cols)
        datasets[name] = df  # Cập nhật lại DataFrame sau khi loại bỏ
    else:
        print(" - Không có cột nào trong danh sách cần loại bỏ.")

    print(f" - Kích thước mới: {df.shape[0]:,} rows × {df.shape[1]} columns")

print("\nHOÀN TẤT VIỆC LOẠI BỎ CÁC CỘT KHÔNG CẦN THIẾT.")



BẮT ĐẦU LOẠI BỎ CÁC CỘT KHÔNG CẦN THIẾT...


>>> SALES ITEM
 - Các cột sẽ bị loại bỏ: ['description', 'manufacturer', 'created_date', 'updated_date', 'created_date', 'updated_date', 'created_date', 'updated_date']
 - Kích thước mới: 27,332 rows × 17 columns

>>> SALES PURCHASE
 - Các cột sẽ bị loại bỏ: ['created_date', 'updated_date', 'created_date', 'updated_date', 'created_date', 'updated_date']
 - Kích thước mới: 35,729,825 rows × 9 columns

>>> SALES USER
 - Các cột sẽ bị loại bỏ: ['created_date', 'updated_date', 'created_date', 'updated_date', 'created_date', 'updated_date']
 - Kích thước mới: 4,573,964 rows × 10 columns

HOÀN TẤT VIỆC LOẠI BỎ CÁC CỘT KHÔNG CẦN THIẾT.


TASK 2.1: NULL

TASK 2.2: OUTLIER


>>> Đang xử lý nhóm SALES ITEM
 - category_l1: null -> Khác
 - category_l2: null -> Khác
 - category_l3: null -> Khác
 - category: null -> Khác
 - description: null -> Không xác định
 - brand: null -> Không rõ
 - manufacturer: null -> Không rõ
 - created_date: null -> 1970-01-01
 - gender_target: null -> Unisex
 - age_group: null -> Không xác định
 - item_type: null -> Khác
 - color: null -> Không xác định
 - size: null -> Free size
 - origin: null -> Không rõ
 - volume: null -> Không xác định
 - material: null -> Không rõ

Null count sau khi điền:
                    0
price               0
category_l1         0
category_l2         0
category_l3         0
category            0
description         0
brand               0
manufacturer        0
creation_timestamp  0
created_date        0
updated_date        0
gender_target       0
age_group           0
item_type           0
gp                  0
color               0
size                0
origin              0
volume              0
mate

TASK 3:

TASK 4:

TASK 5:

TEST
IN RA DU LIEU

In [75]:
# In ra Test
# Lặp qua từng nhóm dữ liệu trong dictionary 'datasets'
for name, df in datasets.items():
    
    # Nếu không có dữ liệu (None), bỏ qua nhóm đó
    if df is None:
        continue

    # In tiêu đề nhóm dữ liệu
    print(f"\n[ {name} ]")

    # In kích thước của DataFrame: số dòng và số cột
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

    # In danh sách tên các cột
    print("Columns:", df.columns)

    # Kiểm tra số lượng giá trị thiếu (null) trong từng cột
    print("\nMissing values:")
    null_counts = df.null_count().to_pandas().T
    print(null_counts)

    # Mô tả thống kê cho các cột số
    print("\nMô tả dữ liệu số:")
    display(df.describe())


[ SALES ITEM ]
Shape: 27,332 rows × 17 columns
Columns: ['price', 'category_l1', 'category_l2', 'category_l3', 'category', 'brand', 'creation_timestamp', 'gender_target', 'age_group', 'item_type', 'gp', 'color', 'size', 'origin', 'volume', 'material', 'sale_status']

Missing values:
                    0
price               0
category_l1         0
category_l2         0
category_l3         0
category            0
brand               0
creation_timestamp  0
gender_target       0
age_group           0
item_type           0
gp                  0
color               0
size                0
origin              0
volume              0
material            0
sale_status         0

Mô tả dữ liệu số:


statistic,price,category_l1,category_l2,category_l3,category,brand,creation_timestamp,gender_target,age_group,item_type,gp,color,size,origin,volume,material,sale_status
str,f64,str,str,str,str,str,f64,str,str,str,f64,str,str,str,str,str,f64
"""count""",27332.0,"""27332""","""27332""","""27332""","""27332""","""27332""",27332.0,"""27332""","""27332""","""27332""",27332.0,"""27332""","""27332""","""27332""","""27332""","""27332""",27332.0
"""null_count""",0.0,"""0""","""0""","""0""","""0""","""0""",0.0,"""0""","""0""","""0""",0.0,"""0""","""0""","""0""","""0""","""0""",0.0
"""mean""",190456.829467,null,null,null,null,null,1.6225e9,null,null,null,49696.651226,null,null,null,null,null,0.167203
"""std""",511123.971813,null,null,null,null,null,7.5666e7,null,null,null,144155.68753,null,null,null,null,null,0.373164
"""min""",0.0,"""Babycare""","""0-1Y""","""A2 milk""","""0-12M Bodysuit bé trai đùi""","""137 Degrees""",1.2983e9,"""Bé Gái""","""0-10M""","""BCS""",0.0,"""Bạc""","""0 tháng""",""" Singapore, Thụy Sĩ""","""1 lít""",""" Vải không dệt, bông cellulose…",0.0
"""25%""",49000.0,null,null,null,null,null,1.5755e9,null,null,null,0.0,null,null,null,null,null,0.0
"""50%""",119000.0,null,null,null,null,null,1.6358e9,null,null,null,25047.0,null,null,null,null,null,0.0
"""75%""",199000.0,null,null,null,null,null,1.6699e9,null,null,null,65532.0,null,null,null,null,null,0.0
"""max""",2.099e7,"""Đồ chơi & Sách""","""Đồ uống""","""Đồ hộp""","""Động vật mô hình""","""Đức Thành""",1.7559e9,"""Unisex""","""[""Từ 6M"", ""Từ 9M"", ""Từ 1Y""]""","""Ứng dụng mẹ và bé""",5.54136e6,"""Đỏ""","""XXL(15-25kg) - 26 miếng""","""Ấn Độ, Bulgaria""","""hộp 3 lọ x 250ml""","""﻿100% cotton﻿<br>""",1.0



[ SALES PURCHASE ]
Shape: 35,729,825 rows × 9 columns
Columns: ['timestamp', 'event_value', 'price', 'date_key', 'quantity', 'channel', 'payment', 'location', 'discount']

Missing values:
             0
timestamp    0
event_value  0
price        0
date_key     0
quantity     0
channel      0
payment      0
location     0
discount     0

Mô tả dữ liệu số:


statistic,timestamp,event_value,price,date_key,quantity,channel,payment,location,discount
str,f64,f64,f64,f64,f64,str,str,f64,f64
"""count""",3.5729825e7,3.5729825e7,3.5729825e7,3.5729825e7,3.5729825e7,"""35729825""","""35729825""",3.5729825e7,3.5729825e7
"""null_count""",0.0,0.0,0.0,0.0,0.0,"""0""","""0""",0.0,0.0
"""mean""",1.7199e9,1.508098,169188.742494,2.0241e7,1.508098,null,null,448.414508,17235.651914
"""std""",9.1568e6,2.264976,195539.88976,346.231465,2.264976,null,null,247.505432,46673.708499
"""min""",1.7041e9,1.0,0.0444,2.0240101e7,1.0,"""Android""","""Chuyển khoản""",16.0,0.0
"""25%""",1.7120e9,1.0,48316.3914,2.0240401e7,1.0,null,null,234.0,0.0
"""50%""",1.7199e9,1.0,87000.0,2.0240702e7,1.0,null,null,439.0,0.0
"""75%""",1.7279e9,1.0,244000.0,2.0241002e7,1.0,null,null,653.0,15000.0
"""max""",1.7357e9,240.0,2.099e7,2.0241231e7,240.0,"""iOS""","""ZaloPay""",1036.0,7.995e6



[ SALES USER ]
Shape: 4,573,964 rows × 10 columns
Columns: ['gender', 'location', 'province', 'membership', 'timestamp', 'region', 'location_name', 'install_app', 'install_date', 'district']

Missing values:
               0
gender         0
location       0
province       0
membership     0
timestamp      0
region         0
location_name  0
install_app    0
install_date   0
district       0

Mô tả dữ liệu số:


statistic,gender,location,province,membership,timestamp,region,location_name,install_app,install_date,district
str,str,f64,str,str,f64,str,str,str,f64,str
"""count""","""4573964""",4.573964e6,"""4573964""","""4573964""",4.573964e6,"""4573964""","""4573964""","""4573964""",4.573964e6,"""4573964"""
"""null_count""","""0""",0.0,"""0""","""0""",0.0,"""0""","""0""","""0""",0.0,"""0"""
"""mean""",null,554.053518,null,null,1.6712e9,null,null,null,1.6841e9,null
"""std""",null,295.523361,null,null,7.9743e7,null,null,null,7.1374e7,null
"""min""","""Khác""",42.0,"""An Giang""","""Diamond""",1.3064e9,"""Bắc Trung Bộ""","""AGI - 110 Quốc Lộ 91""","""Android""",1.3063e9,""" Vũng Liêm"""
"""25%""",null,302.0,null,null,1.6343e9,null,null,null,1.6550e9,null
"""50%""",null,547.0,null,null,1.6960e9,null,null,null,1.7054e9,null
"""75%""",null,765.0,null,null,1.7312e9,null,null,null,1.7366e9,null
"""max""","""Nữ""",1310.0,"""Đồng Tháp""","""Standard""",1.7593e9,"""Đồng bằng sông Cửu Long""","""ĐTH - Trần Hưng Đạo""","""iOS""",1.7592e9,"""Ứng Hòa"""
